In [1]:
import os
from code.config import args 

# 句子切割

## _cut

In [2]:
def _cut(sentence):
    """
    将一段文本切分成多个句子
    :param sentence:
    :return:
    """
    new_sentence = []
    sen = []
    for i in sentence:
        if i in ['。', '！', '？', '?'] and len(sen) != 0:
            sen.append(i)
            new_sentence.append("".join(sen))
            sen = []
            continue
        sen.append(i)

    if len(new_sentence) <= 1: # 一句话超过max_seq_length且没有句号的，用","分割，再长的不考虑了。
        new_sentence = []
        sen = []
        for i in sentence:
            if i.split(' ')[0] in ['，', ','] and len(sen) != 0:
                sen.append(i)
                new_sentence.append("".join(sen))
                sen = []
                continue
            sen.append(i)
    if len(sen) > 0:  # 若最后一句话无结尾标点，则加入这句话
        new_sentence.append("".join(sen))
    return new_sentence



## cut_test_set

In [3]:
def cut_test_set(text_list,len_treshold):
    cut_text_list = []
    cut_index_list = []
    for text in text_list:

        temp_cut_text_list = []
        text_agg = ''
        if len(text) < len_treshold:
            temp_cut_text_list.append(text)
        else:
            sentence_list = _cut(text)  # 一条数据被切分成多句话
            for sentence in sentence_list:
                if len(text_agg) + len(sentence) < len_treshold:
                    text_agg += sentence
                else:
                    temp_cut_text_list.append(text_agg)
                    text_agg = sentence
            temp_cut_text_list.append(text_agg)  # 加上最后一个句子

        cut_index_list.append(len(temp_cut_text_list))
        cut_text_list += temp_cut_text_list

    return cut_text_list, cut_index_list


# 数据处理

In [4]:
#设置样本长度
text_length = 250
def from_ann2dic(r_ann_path, r_txt_path, w_path, w_file):
    """
    r_ann_path: 源ann 文件path
    r_txt_path
    w_path：目标目录
    w_file：文件名称
    """
    q_dic = {}
    with codecs.open(r_ann_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip("\n\r")
            line_arr = line.split('\t')
            entityinfo = line_arr[1]
            entityinfo = entityinfo.split(' ')
            cls = entityinfo[0]
            start_index = int(entityinfo[1])
            end_index = int(entityinfo[2])
            length = end_index - start_index
            for r in range(length):
                if r == 0:
                    q_dic[start_index] = ("B-%s" % cls)
                else:
                    q_dic[start_index + r] = ("I-%s" % cls)

    with codecs.open(r_txt_path, "r", encoding="utf-8") as f:
        content_str = f.read()
        
    
    cut_text_list, cut_index_list = cut_test_set([content_str],text_length)
    
    i = 0
    for idx, line in enumerate(cut_text_list):
        w_path_ = "%s/%s-%s-new.txt" % (w_path, w_file,idx)
        with codecs.open(w_path_, "w", encoding="utf-8") as w:
            for str_ in line:
                if str_ == " " or str_ == "" or str_ == "\n" or str_ == "\r":
                    pass
                else:
                    if i in q_dic:
                        tag = q_dic[i]
                    else:
                        tag = "O"  # 大写字母O
                    w.write('%s %s\n' % (str_, tag))
                i+=1
            w.write('%s\n' % "END O")    

# 划分训练集和验证集

In [5]:
import glob
file_list = glob.glob(f'{args.DATA_DIR}train/*.txt') 

In [8]:
file_list[:5]

['../../../mydata/data_origin/cndrug_ner/train/63.txt',
 '../../../mydata/data_origin/cndrug_ner/train/256.txt',
 '../../../mydata/data_origin/cndrug_ner/train/304.txt',
 '../../../mydata/data_origin/cndrug_ner/train/728.txt',
 '../../../mydata/data_origin/cndrug_ner/train/456.txt']

In [7]:
from sklearn.model_selection import train_test_split

In [9]:
train_filelist, val_filelist = train_test_split(file_list,test_size=0.2,random_state=666)

In [10]:
len(train_filelist)

800

In [11]:
len(val_filelist)

200

In [12]:
f'{args.DATA_GEN}train'

'../../../mydata/data_gen/cndrug_ner/train'

In [18]:
!mkdir  ../../../mydata/data_gen/cndrug_ner/train
!mkdir ../../../mydata/data_gen/cndrug_ner/val

# 训练集处理

In [19]:
import os
import codecs
data_dir = f'{args.DATA_DIR}train/'
for file in train_filelist:
    if file.find(".ann") == -1 and file.find(".txt") == -1:
        continue
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = f'{args.DATA_GEN}train'
    w_file = file_name
    from_ann2dic(r_ann_path, r_txt_path, w_path,w_file)


# 验证集处理

In [22]:
import os
import codecs
data_dir = f'{args.DATA_DIR}train/'
for file in val_filelist:
    if file.find(".ann") == -1 and file.find(".txt") == -1:
        continue
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = f'{args.DATA_GEN}val'
    w_file = file_name
    from_ann2dic(r_ann_path, r_txt_path, w_path,w_file)


# 训练集合并

In [24]:
os.listdir(f'{args.DATA_DIR}train/')

['808.ann',
 '63.txt',
 '601.ann',
 '256.txt',
 '304.txt',
 '870.ann',
 '728.txt',
 '456.txt',
 '39.txt',
 '899.ann',
 '955.txt',
 '337.txt',
 '506.txt',
 '473.txt',
 '509.txt',
 '355.ann',
 '590.ann',
 '92.ann',
 '392.ann',
 '228.ann',
 '365.ann',
 '599.ann',
 '685.ann',
 '801.txt',
 '455.txt',
 '850.ann',
 '416.ann',
 '64.ann',
 '669.txt',
 '852.ann',
 '566.ann',
 '838.txt',
 '834.txt',
 '297.txt',
 '860.ann',
 '466.txt',
 '672.txt',
 '827.txt',
 '653.ann',
 '542.ann',
 '551.txt',
 '225.txt',
 '618.ann',
 '128.txt',
 '348.txt',
 '140.txt',
 '429.txt',
 '606.ann',
 '956.txt',
 '923.ann',
 '816.txt',
 '931.txt',
 '313.ann',
 '393.ann',
 '518.txt',
 '554.txt',
 '775.ann',
 '933.ann',
 '121.txt',
 '493.ann',
 '958.txt',
 '808.txt',
 '935.txt',
 '854.ann',
 '922.txt',
 '812.ann',
 '241.ann',
 '997.txt',
 '32.ann',
 '786.txt',
 '150.ann',
 '681.txt',
 '515.ann',
 '886.txt',
 '987.txt',
 '120.txt',
 '973.ann',
 '123.ann',
 '537.ann',
 '154.ann',
 '558.ann',
 '106.txt',
 '677.ann',
 '278.txt

In [27]:
w_path = f"{args.DATA_GEN}train.txt"
for file in os.listdir(f'{args.DATA_GEN}train/'):
    path = os.path.join(f'{args.DATA_GEN}train/', file)
    if not file.endswith(".txt"):  
        continue
    q_list = []
    print("开始读取文件:%s" % file)
    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END O":
#             print(line)
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    print("开始写入文本%s" % w_path)
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()

开始读取文件:583-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:695-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:806-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:866-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:962-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:699-2-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:223-2-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:954-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:118-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:96-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:666-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:937-2-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:315-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:953-1-new.txt
开始写入文本../../../mydata/data_gen/

开始读取文件:267-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:581-4-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:599-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:267-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:313-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:107-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:898-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:293-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:18-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:406-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:784-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:816-2-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:239-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:720-0-new.txt
开始写入文本../../../mydata/data_gen/

开始读取文件:989-2-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:163-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:636-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:322-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:739-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:798-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:646-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:943-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:236-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:721-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:452-2-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:408-1-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:880-0-new.txt
开始写入文本../../../mydata/data_gen/cndrug_ner/train.txt
开始读取文件:134-2-new.txt
开始写入文本../../../mydata/data_gen

# 验证集合并

In [28]:
w_path = f"{args.DATA_GEN}val.txt"
for file in os.listdir(f'{args.DATA_GEN}val/'):
    path = os.path.join(f'{args.DATA_GEN}val/', file)
    if not file.endswith(".txt"):  
        continue
    q_list = []

    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END O":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()

# 原始验证集拷贝

In [30]:
val_filelist

['../../../mydata/data_origin/cndrug_ner/train/428.txt',
 '../../../mydata/data_origin/cndrug_ner/train/383.txt',
 '../../../mydata/data_origin/cndrug_ner/train/50.txt',
 '../../../mydata/data_origin/cndrug_ner/train/180.txt',
 '../../../mydata/data_origin/cndrug_ner/train/291.txt',
 '../../../mydata/data_origin/cndrug_ner/train/382.txt',
 '../../../mydata/data_origin/cndrug_ner/train/938.txt',
 '../../../mydata/data_origin/cndrug_ner/train/812.txt',
 '../../../mydata/data_origin/cndrug_ner/train/240.txt',
 '../../../mydata/data_origin/cndrug_ner/train/803.txt',
 '../../../mydata/data_origin/cndrug_ner/train/924.txt',
 '../../../mydata/data_origin/cndrug_ner/train/531.txt',
 '../../../mydata/data_origin/cndrug_ner/train/563.txt',
 '../../../mydata/data_origin/cndrug_ner/train/908.txt',
 '../../../mydata/data_origin/cndrug_ner/train/573.txt',
 '../../../mydata/data_origin/cndrug_ner/train/299.txt',
 '../../../mydata/data_origin/cndrug_ner/train/648.txt',
 '../../../mydata/data_origin/cn

In [35]:
!mkdir ../../../mydata/data_gen/cndrug_ner/val_data

In [34]:
f"{args.DATA_GEN}val_data"

'../../../mydata/data_gen/cndrug_ner/val_data'

In [36]:
for file in val_filelist:
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join(f"{args.DATA_DIR}train", "%s.ann" % file_name)
    os.system("cp %s %s"%(file,f"{args.DATA_GEN}val_data/"))
    os.system("cp %s %s"%(r_ann_path,f"{args.DATA_GEN}val_data/"))
    print(file)

../../../mydata/data_origin/cndrug_ner/train/428.txt
../../../mydata/data_origin/cndrug_ner/train/383.txt
../../../mydata/data_origin/cndrug_ner/train/50.txt
../../../mydata/data_origin/cndrug_ner/train/180.txt
../../../mydata/data_origin/cndrug_ner/train/291.txt
../../../mydata/data_origin/cndrug_ner/train/382.txt
../../../mydata/data_origin/cndrug_ner/train/938.txt
../../../mydata/data_origin/cndrug_ner/train/812.txt
../../../mydata/data_origin/cndrug_ner/train/240.txt
../../../mydata/data_origin/cndrug_ner/train/803.txt
../../../mydata/data_origin/cndrug_ner/train/924.txt
../../../mydata/data_origin/cndrug_ner/train/531.txt
../../../mydata/data_origin/cndrug_ner/train/563.txt
../../../mydata/data_origin/cndrug_ner/train/908.txt
../../../mydata/data_origin/cndrug_ner/train/573.txt
../../../mydata/data_origin/cndrug_ner/train/299.txt
../../../mydata/data_origin/cndrug_ner/train/648.txt
../../../mydata/data_origin/cndrug_ner/train/269.txt
../../../mydata/data_origin/cndrug_ner/train/60

../../../mydata/data_origin/cndrug_ner/train/348.txt
../../../mydata/data_origin/cndrug_ner/train/619.txt
../../../mydata/data_origin/cndrug_ner/train/923.txt
../../../mydata/data_origin/cndrug_ner/train/161.txt
../../../mydata/data_origin/cndrug_ner/train/70.txt
../../../mydata/data_origin/cndrug_ner/train/711.txt
../../../mydata/data_origin/cndrug_ner/train/271.txt
../../../mydata/data_origin/cndrug_ner/train/727.txt
../../../mydata/data_origin/cndrug_ner/train/837.txt
../../../mydata/data_origin/cndrug_ner/train/481.txt
../../../mydata/data_origin/cndrug_ner/train/679.txt
../../../mydata/data_origin/cndrug_ner/train/123.txt
../../../mydata/data_origin/cndrug_ner/train/788.txt
../../../mydata/data_origin/cndrug_ner/train/821.txt
../../../mydata/data_origin/cndrug_ner/train/750.txt
../../../mydata/data_origin/cndrug_ner/train/662.txt
../../../mydata/data_origin/cndrug_ner/train/371.txt
../../../mydata/data_origin/cndrug_ner/train/594.txt
../../../mydata/data_origin/cndrug_ner/train/33